### Fine tune RoBerta with LoRA

In [ ]:
!pip install -q transformers 
!pip install -q peft
!pip install -q evaluate
!pip install wandb

### 1. Injecting LoRA to the RoBerta model

In [2]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    # r: the dimension of the A and B
    # lora_alpha: a scaling factor -> the relative significance of the weights 
    #                                 in ‘A’ and ‘B’ in relation to the model’s 
    #                                 original parameters
    task_type=TaskType.SEQ_CLS, r=8, lora_alpha=8, lora_dropout=0.1
)

In [3]:
import torch
from transformers import RobertaForSequenceClassification

# get RoBerta model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained('roberta-base').to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# insert the ‘A’ and ‘B’ matrices into our model by invoking the ‘get_peft_model’ function.
from peft import get_peft_model

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066137476531099


### 2. Get Datasets

In [5]:
from nli_dataset import NliDataset

# get dataset
train_dataset = NliDataset(csv_file="Data/train_data.csv", max_length=256)
val_dataset = NliDataset(csv_file="Data/test_data.csv", max_length=256)


In [6]:
from torch.utils.data import DataLoader
train_loader = DataLoader(
                        dataset=train_dataset,
                        batch_size=64,
                        num_workers=4,
                        prefetch_factor=2,
                        shuffle=True,
                        drop_last=False
                    )

val_loader = DataLoader(
                    dataset=val_dataset,
                    batch_size=64,
                    num_workers=4,
                    prefetch_factor=2,
                    drop_last=False
                )

### Train

In [7]:
from torch.optim import lr_scheduler
from transformers import AdamW

# Define optimizer and loss function
optimizer = AdamW(model.parameters(),
                betas=(0.9, 0.98),  # according to RoBERTa paper
                lr=1e-4, 
                weight_decay=5e-2)

linear_sl = lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=1)
cos_sl = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
scheduler = lr_scheduler.SequentialLR(optimizer, schedulers=[linear_sl, cos_sl], milestones=[2])

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

c:\Users\wzy\anaconda3\envs\nlu\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
import wandb

wandb.init(
    # set the wandb project where this run will be logged
    project="NLI-Roberta-LoRA",

    # track hyperparameters and run metadata
    config={
        'learning_rate': 1e-4,
        'optimizer': 'AdamW',
        'batch_size': 64,
        'epochs': 20
    }
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wziyi1169 (ziyiwang). Use `wandb login --relogin` to force relogin


In [9]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Training loop
total_best_val_acc = 0
for epoch in range(20):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fn(outputs.logits, labels)
        train_loss += loss.item()
        # loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
        optimizer.step()
    
    scheduler.step()
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss / len(train_loader)}')
    
    # Validation loop
    model.eval()
    val_loss = 0
    # Initialize lists to store true labels and predicted labels
    true_labels_list = []
    preds_list = []
    for batch in tqdm(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = loss_fn(outputs.logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)

            # Append true labels and predicted labels to the lists
            true_labels_list.extend(labels.cpu().numpy())
            preds_list.extend(preds.cpu().numpy())

    # Calculate accuracy and F1-score
    # Calculate evaluation metrics
    val_acc = accuracy_score(true_labels_list, preds_list)
    f1 = f1_score(true_labels_list, preds_list, average='weighted')
    precision = precision_score(true_labels_list, preds_list, average='weighted')
    recall = recall_score(true_labels_list, preds_list, average='weighted')

    wandb.log({"Epoch":epoch + 1, 
                "Train_Loss": train_loss / len(train_loader),
                "Val_Loss": val_loss / len(val_loader),
                "Val_Acc": val_acc,
                "F1": f1,
                "Precision": precision,
                "Recall": recall
            })

    if val_acc > total_best_val_acc:
        total_best_val_acc = val_acc
        torch.save(model.state_dict(), 'roberta_lora/best_model.pth')
    print(f'Validation: Epoch {epoch + 1}, Loss: {val_loss / len(val_loader)}, Acc: {val_acc}, F1: {f1}, Precision: {precision}, Recall: {recall}')

100%|██████████| 337/337 [07:17<00:00,  1.30s/it]


Epoch 1, Train Loss: 0.6952152584709824


100%|██████████| 85/85 [00:28<00:00,  3.03it/s]
c:\Users\wzy\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation: Epoch 1, Loss: 0.6921639470493093, Acc: 0.5227314900723696, F1: 0.35889217829243614, Precision: 0.27324821071327987, Recall: 0.5227314900723696


100%|██████████| 337/337 [07:16<00:00,  1.30s/it]


...

Validation: Epoch 19, Loss: 0.3898716975660885, Acc: 0.8784561143069215, F1: 0.8784723652512237, Precision: 0.8785061183017059, Recall: 0.8784561143069215


100%|██████████| 337/337 [07:17<00:00,  1.30s/it]


Epoch 20, Train Loss: 0.4008827855926593


100%|██████████| 85/85 [00:27<00:00,  3.04it/s]

Validation: Epoch 20, Loss: 0.388645177378374, Acc: 0.8791983670439785, F1: 0.8791295195138855, Precision: 0.8792444567064556, Recall: 0.8791983670439785


In [10]:
wandb.finish()

Epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
F1,▁▇██████████████████
Precision,▁▇██████████████████
Recall,▁▇▇▇████████████████
Train_Loss,█▆▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Val_Acc,▁▇▇▇████████████████
Val_Loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,20
F1,0.87913
Precision,0.87924
Recall,0.8792
